<a href="https://colab.research.google.com/github/opassos/FastAI-Exemplos/blob/main/FastAI%20Model%20Splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -Uqqq fastai
! pip install -Uqqq timm

     |████████████████████████████████| 376 kB 5.1 MB/s 


In [ ]:
from fastai.vision.all import *
import timm

In [ ]:
set_seed(99, True)
path = untar_data(URLs.PETS)/'images'
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2,
    label_func=lambda x: x[0].isupper(), item_tfms=Resize(224))

In [ ]:
def uname_model(m):
    l = []
    children = list(m.children())
    for c in children:
        if list(c.children()):
            l.append(uname_model(c))
        else:
            l.append(c)
    return nn.Sequential(*l)

def flatten_model(m):
    l = []
    children = list(m.children())
    for c in children:
        if list(c.children()):
            l.extend(flatten_model(c))
        else:
            l.append(c)
    return nn.Sequential(*l)

In [148]:
class print_frozen(Callback):
    def before_fit(self):
        total = sum([np.prod(p.size()) for p in self.model.parameters()])
        frozen = filter(lambda p: not p.requires_grad, self.model.parameters())
        frozen = sum([np.prod(p.size()) for p in frozen])
        print(f'Model training {total - frozen} parameters out of {total} ({(100*(total - frozen)/total):.2f}%)')

In [ ]:
model = resnet18()
model

In [181]:
def my_split(m): return L(*m).map(params)
learn = Learner(dls, model, splitter = Pipeline([uname_model, my_split]), lr = slice(1, 10), cbs = print_frozen)
learn.freeze()
learn.print_frozen('before_fit')
L([f"{x['lr']:.2f}" for x in learn.opt.hypers])

Model training 522600 parameters out of 11689512 (4.47%)


(#10) ['1.00','1.29','1.67','2.15','2.78','3.59','4.64','5.99','7.74','10.00']

In [187]:
def my_split(m): return L(*m[:40], m[40:]).map(params)
learn = Learner(dls, model, splitter = Pipeline([flatten_model, my_split]), lr = slice(1, 10), cbs = print_frozen)
learn.freeze()
learn.print_frozen('before_fit')
L([f"{x['lr']:.2f}" for x in learn.opt.hypers])

Model training 7731560 parameters out of 11689512 (66.14%)


(#41) ['1.00','1.06','1.12','1.19','1.26','1.33','1.41','1.50','1.58','1.68'...]

In [189]:
def my_split(m): return L(m[:6], m[6:-3], m[-3:]).map(params)
learn = Learner(dls, model, splitter = Pipeline([uname_model, my_split]), lr = slice(1, 10), cbs = print_frozen)
learn.freeze()
learn.print_frozen('before_fit')
L([f"{x['lr']:.2f}" for x in learn.opt.hypers])

Model training 8911208 parameters out of 11689512 (76.23%)


(#3) ['1.00','3.16','10.00']

In [191]:
def my_split(m): return L(m[:6], *flatten_model(m[6:-3]), m[-3:]).map(params)
learn = Learner(dls, model, splitter = Pipeline([uname_model, my_split]), lr = slice(1, 10), cbs = print_frozen)
learn.freeze()
learn.print_frozen('before_fit')
L([f"{x['lr']:.2f}" for x in learn.opt.hypers])

Model training 8911208 parameters out of 11689512 (76.23%)


(#14) ['1.00','1.19','1.43','1.70','2.03','2.42','2.89','3.46','4.12','4.92'...]

In [193]:
class literal_split():
    def __init__(self, split_string, unwrapper = uname_model):
        self.s = split_string
        self.unwrapper = unwrapper

    def __call__(self, m):
        m = self.unwrapper(m)
        return eval('L(' + self.s + ')').map(params)

In [204]:
splitter = literal_split('m[:6], *flatten_model(m[6:-3]), m[-3:]')
learn = Learner(dls, model, splitter = splitter, lr = slice(1, 10), cbs = print_frozen)
learn.freeze()
learn.print_frozen('before_fit')
L([f"{x['lr']:.2f}" for x in learn.opt.hypers])

Model training 8911208 parameters out of 11689512 (76.23%)


(#14) ['1.00','1.19','1.43','1.70','2.03','2.42','2.89','3.46','4.12','4.92'...]